In [24]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/english-dataset/eng_final_train.csv
/kaggle/input/english-dataset/eng_final_test.csv


In [25]:
# !pip install --upgrade --force-reinstall huggingface-hub



In [26]:
# !pip install --upgrade transformers huggingface_hub


In [27]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

import torch
import torch.nn as nn
from transformers import RobertaTokenizer, RobertaModel

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras import utils
from keras import backend as K

In [28]:
# ----------------------
# 1. Load Data
# ----------------------
train_df = pd.read_csv("/kaggle/input/english-dataset/eng_final_train.csv")
test_df = pd.read_csv("/kaggle/input/english-dataset/eng_final_test.csv")

train_df = train_df.sample(frac=1).reset_index(drop=True)
test_df = test_df.sample(frac=1).reset_index(drop=True)

In [29]:
from huggingface_hub import snapshot_download

# Downloads model locally
local_dir = snapshot_download(repo_id="roberta-base")
print(local_dir)





Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/445 [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

flax_model.msgpack:   0%|          | 0.00/499M [00:00<?, ?B/s]

...

tf_model.h5:   0%|          | 0.00/657M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

rust_model.ot:   0%|          | 0.00/656M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

/root/.cache/huggingface/hub/models--roberta-base/snapshots/e2da8e2f811d1448a5b465c236feacd80ffbac7b


In [30]:
# ----------------------
# 3. Extract Embeddings
# ----------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MODEL_DIR = "/root/.cache/huggingface/hub/models--roberta-base/snapshots/e2da8e2f811d1448a5b465c236feacd80ffbac7b"

tokenizer = RobertaTokenizer.from_pretrained(MODEL_DIR)
model = RobertaModel.from_pretrained(MODEL_DIR, output_hidden_states=True)
model = model.to(device)
model.eval()

def get_roberta_embeddings(texts, max_len=512):
    embeddings = []
    for sent in tqdm(texts):
        encoded = tokenizer.encode_plus(
            sent,
            add_special_tokens=True,
            max_length=max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        input_ids = encoded['input_ids'].to(device)
        attention_mask = encoded['attention_mask'].to(device)

        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            hidden_states = outputs.hidden_states
            # Mean pooling of second-to-last layer
            sent_emb = torch.mean(hidden_states[-2], dim=1)
            embeddings.append(sent_emb.cpu().numpy())
    return np.vstack(embeddings)

# Compute embeddings
train_embeds = get_roberta_embeddings(train_df['text'])
test_embeds = get_roberta_embeddings(test_df['text'])


Some weights of RobertaModel were not initialized from the model checkpoint at /root/.cache/huggingface/hub/models--roberta-base/snapshots/e2da8e2f811d1448a5b465c236feacd80ffbac7b and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 2559/2559 [01:24<00:00, 30.32it/s]


In [31]:
# -----------------------------
# 4. Save Embeddings (optional)
# -----------------------------
np.save('train_roberta_embeds.npy', train_embeds)
np.save('test_roberta_embeds.npy', test_embeds)

In [32]:
# -----------------------------
# 5. Load Embeddings
# -----------------------------
train_embeds = np.load('train_roberta_embeds.npy')
test_embeds = np.load('test_roberta_embeds.npy')

# Make sure labels are numpy arrays
y_train = train_df['tag'].to_numpy()
y_test = test_df['tag'].to_numpy()

In [35]:
# -----------------------------
# 4. Save Embeddings (optional)
# -----------------------------
np.save('train_roberta_embeds.npy', train_embeds)
np.save('test_roberta_embeds.npy', test_embeds)

# -----------------------------
# 5. Load Embeddings
# -----------------------------
train_embeds = np.load('train_roberta_embeds.npy')
test_embeds = np.load('test_roberta_embeds.npy')

# Make sure labels are numpy arrays
y_train = train_df['tag'].to_numpy()
y_test = test_df['tag'].to_numpy()

# -----------------------------
# 6. Train Classifiers (including DNN)
# -----------------------------
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras import utils

classifier_list = [
    LogisticRegression(max_iter=500),
    RandomForestClassifier(n_estimators=100),
    SVC(probability=True)
]

for clf in classifier_list:
    print(f"\nTraining {clf} ...")
    
    
    
        # For sklearn classifiers
    clf.fit(train_embeds, y_train)
    y_pred_labels = clf.predict(test_embeds)
    y_test_labels = y_test
    
    # Metrics
    cm = confusion_matrix(y_test_labels, y_pred_labels)
    print(f"Confusion Matrix:\n{cm}")
    cr = classification_report(y_test_labels, y_pred_labels, target_names=['no-hope', 'hope'])
    print(f"Classification Report:\n{cr}")



Training LogisticRegression(max_iter=500) ...


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Confusion Matrix:
[[2300   36]
 [ 162   61]]
Classification Report:
              precision    recall  f1-score   support

     no-hope       0.93      0.98      0.96      2336
        hope       0.63      0.27      0.38       223

    accuracy                           0.92      2559
   macro avg       0.78      0.63      0.67      2559
weighted avg       0.91      0.92      0.91      2559


Training RandomForestClassifier() ...
Confusion Matrix:
[[2329    7]
 [ 202   21]]
Classification Report:
              precision    recall  f1-score   support

     no-hope       0.92      1.00      0.96      2336
        hope       0.75      0.09      0.17       223

    accuracy                           0.92      2559
   macro avg       0.84      0.55      0.56      2559
weighted avg       0.91      0.92      0.89      2559


Training SVC(probability=True) ...
Confusion Matrix:
[[2336    0]
 [ 223    0]]
Classification Report:
              precision    recall  f1-score   support

     no-hope

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [36]:
# -----------------------------
# DNN Training and Evaluation
# -----------------------------
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, Input
from keras import utils
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf

def recall_m(y_true, y_pred):
    true_positives = tf.reduce_sum(tf.round(tf.clip_by_value(y_true * y_pred, 0, 1)))
    possible_positives = tf.reduce_sum(tf.round(tf.clip_by_value(y_true, 0, 1)))
    recall = true_positives / (possible_positives + tf.keras.backend.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = tf.reduce_sum(tf.round(tf.clip_by_value(y_true * y_pred, 0, 1)))
    predicted_positives = tf.reduce_sum(tf.round(tf.clip_by_value(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + tf.keras.backend.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+tf.keras.backend.epsilon()))


# -----------------------------
# Load embeddings
# -----------------------------
train_embeds = np.load('train_roberta_embeds.npy')
test_embeds = np.load('test_roberta_embeds.npy')

y_train = train_df['tag'].to_numpy()
y_test = test_df['tag'].to_numpy()

# -----------------------------
# Prepare DNN input
# -----------------------------
X_train_dnn = np.reshape(train_embeds, (train_embeds.shape[0], 1, train_embeds.shape[1]))
X_test_dnn = np.reshape(test_embeds, (test_embeds.shape[0], 1, test_embeds.shape[1]))

y_train_dnn = utils.to_categorical(y_train, 2)
y_test_dnn = utils.to_categorical(y_test, 2)

# -----------------------------
# Build DNN Model
# -----------------------------
model_dnn = Sequential()
model_dnn.add(Input(shape=(1, X_train_dnn.shape[2])))
model_dnn.add(LSTM(100))
model_dnn.add(Dense(64, activation='relu'))
model_dnn.add(Dense(2, activation='softmax'))

model_dnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', f1_m])

# -----------------------------
# Train DNN
# -----------------------------
model_dnn.fit(X_train_dnn, y_train_dnn, epochs=3, batch_size=8, validation_split=0.1)

# -----------------------------
# Evaluate DNN
# -----------------------------
y_pred = model_dnn.predict(X_test_dnn)
y_pred_labels = np.argmax(y_pred, axis=1)
y_test_labels = np.argmax(y_test_dnn, axis=1)

cm = confusion_matrix(y_test_labels, y_pred_labels)
print(f"Confusion Matrix:\n{cm}")

cr = classification_report(y_test_labels, y_pred_labels, target_names=['no-hope', 'hope'])
print(f"Classification Report:\n{cr}")

# -----------------------------
# Save DNN model (optional)
# -----------------------------
model_dnn.save('dnn_roberta_model.h5')


Epoch 1/3


I0000 00:00:1761208359.550406     145 cuda_dnn.cc:529] Loaded cuDNN version 90300


2590/2590 ━━━━━━━━━━━━━━━━━━━━ 19s 6ms/step - accuracy: 0.9126 - f1_m: 0.9126 - loss: 0.2603 - val_accuracy: 0.9158 - val_f1_m: 0.9157 - val_loss: 0.2167
Epoch 2/3
2590/2590 ━━━━━━━━━━━━━━━━━━━━ 14s 5ms/step - accuracy: 0.9242 - f1_m: 0.9242 - loss: 0.2120 - val_accuracy: 0.9084 - val_f1_m: 0.9083 - val_loss: 0.2337
Epoch 3/3
2590/2590 ━━━━━━━━━━━━━━━━━━━━ 14s 5ms/step - accuracy: 0.9260 - f1_m: 0.9260 - loss: 0.1989 - val_accuracy: 0.9188 - val_f1_m: 0.9187 - val_loss: 0.2140
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Confusion Matrix:
[[2309   27]
 [ 163   60]]
Classification Report:
              precision    recall  f1-score   support

     no-hope       0.93      0.99      0.96      2336
        hope       0.69      0.27      0.39       223

    accuracy                           0.93      2559
   macro avg       0.81      0.63      0.67      2559
weighted avg       0.91      0.93      0.91      2559

